<a href="https://colab.research.google.com/github/SiweiLuo/DL_with_Python/blob/master/TwoModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# NOTE: define the following variables
#       top_model_weights_path
#       num_classes
#       dense_layer_1 = 4096
#       dense_layer_2 = 4096

vgg16 = applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3))

# Inspect the model
vgg16.summary()

# This shape has to match the last layer in VGG16 (without top)
dense_input  = Input(shape=(7, 7, 512))
dense_output = Flatten(name='flatten')(dense_input)
dense_output = Dense(dense_layer_1, activation='relu', name='fc1')(dense_output)
dense_output = Dense(dense_layer_2, activation='relu', name='fc2')(dense_output)
dense_output = Dense(num_classes, activation='softmax', name='predictions')(dense_output)

top_model = Model(inputs=dense_input, outputs=dense_output, name='top_model')

# from: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

block5_pool = vgg16.get_layer('block5_pool').output

# Now combine the two models
full_output = top_model(block5_pool)
full_model  = Model(inputs=vgg16.input, outputs=full_output)

# set the first 15 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
# WARNING: this may not be applicable for Inception V3
for layer in full_model.layers[:15]:
    layer.trainable = False

# Verify things look as expected
full_model.summary()

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
full_model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.SGD(lr=5e-5, momentum=0.9),
    metrics=['accuracy'])

# Train the model...